# epub Creator - wuxiaworld.co

This notebook creates an epub file from a webnovel in https://www.wuxiaworld.co

In [ ]:
!pip install ebooklib requests-html pillow bs4 

In [77]:
import bs4
from ebooklib import epub
from requests_html import HTMLSession

from PIL import Image
import io

import sys

import requests 

# Define the web-novel you want (Variables)

In [79]:
book_title = 'King of Gods'
cover_image_url = ('https://vignette.wikia.nocookie.net/king-of-god/images/9/95/King-of-gods.jpg')
url = 'https://www.wuxiaworld.co/King-of-Gods/'
author = 'Fast Food Resturant ( 快餐店)'

# Download Chapters and create epub file

In [85]:
book = epub.EpubBook()

# set metadata
book.set_identifier('')
book.set_title(book_title)
book.set_language('en')
#r_image = requests.get(cover_image_url)
#!wget https://vignette.wikia.nocookie.net/king-of-god/images/9/95/King-of-gods.jpg

with open('cover.jpg', 'wb') as handle:
        response = requests.get(cover_image_url, stream=True)

        if not response.ok:
            print(response)

        for block in response.iter_content(1024):
            if not block:
                break

            handle.write(block)


# add cover image
book.set_cover("image.jpg", open('cover.jpg', 'rb').read())

book.add_author(author)

In [86]:
toc_list = []
output_stream = sys.stdout

soup = bs4.BeautifulSoup(requests.get(url).content,features='lxml')
for link in soup.findAll('a'):
    if 'style' in str(link):
        chapter = link.get("href")
        r = session.get(url+str(chapter))
        chapter_title = r.html.find('h1')
        chapter = r.html.find('div#content')
        #print(chapter_title[0].text)
        #print(chapter[0].text)
        #print('---------------------------------')
        epub_chapter = epub.EpubHtml(title=u'{}'.format(chapter_title[0].text), file_name='{}.xhtml'.format(chapter_title[0].text.replace(' ','_')), lang='hr')
        epub_chapter.content = chapter[0].html
        book.add_item(epub_chapter)
        toc_list.append(epub_chapter)
        
        sys.stdout.write("\rChapter %s" % chapter_title[0].text)
        sys.stdout.flush()
        

Chapter 1246 Conference

In [87]:
# define Table Of Contents
book.toc = tuple(toc_list)

# add default NCX and Nav file
book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())

# define CSS style
style = 'BODY {color: white;}'
nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)

# add CSS file
book.add_item(nav_css)

# basic spine
book.spine = ['nav']+toc_list

# write to the file
epub.write_epub('{}-wuxiaworld.epub'.format(book_title.replace(' ', '_')), book, {})